In [1]:
import pandas as pd 
import re
pathmoe = '/hdd3/sonia/be_great/ckpts/moe/dgpt2/adult-allcol/jul21/samples'
pathoh = '/hdd3/sonia/be_great/ckpts/dgpt2/adult-allcol/samples'
pathreal = '/hdd3/sonia/data/adult.csv'
pathgreatdgpt2 = '/hdd3/sonia/be_great/ckpts/dgpt2-great'
path = pathmoe

age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income

### From text

In [2]:
patternmoe = r'^\<\|endoftext\|\>age is (\?|\d+)\.<EOS>workclass is (.+)\.<EOS>fnlwgt is (\?|\d+)\.<EOS>education is (.+)\.<EOS>' +\
             r'education-num is (\?|\d+)\.<EOS>marital-status is (.+)\.<EOS>occupation is (.+)\.<EOS>relationship is (.+)\.<EOS>' +\
             r'race is (.+)\.<EOS>sex is (.+)\.<EOS>capital-gain is (\?|\d+)\.<EOS>capital-loss is (\?|\d+)\.<EOS>' +\
             r'hours-per-week is (\?|\d+)\.<EOS>native-country is ([-\w]+)\.<EOS>income is (.+?)\.<EOS>.?'
# patternoh = f'\<\|endoftext\|\>age is (\d+)\.<EOS> workclass is (\w+)\.<EOS> fnlwgt is (\d+).+<EOS> income is (.+)\.<EOS>'
pattern = patternmoe

In [3]:
with open(path+'.txt', 'r') as f:
    raws = f.readlines()

parsed = []
for raw in raws:
    vals = re.findall(pattern, raw)
    if len(vals) == 1:
        parsed.append(vals[0])
len(parsed)

9320

In [4]:
cols = ['age','workclass','fnlwgt','education','education-num','marital-status','occupation','relationship',
        'race','sex','capital-gain','capital-loss','hours-per-week','native-country','income']
df = pd.DataFrame(parsed, columns=cols)
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,53,Private,281231,Masters,14,Married-civ-spouse,Exec-managerial,Husband,White,Male,7298,0,60,Mexico,>50K
1,30,Private,328764,Masters,14,Divorced,Prof-specialty,Unmarried,White,Female,0,0,55,Italy,>50K
2,41,Private,185835,Doctorate,16,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,35,United-States,>50K
3,24,Private,112156,Some-college,10,Never-married,Adm-clerical,Own-child,White,Female,0,0,40,United-States,<=50K
4,25,Private,108425,HS-grad,9,Married-civ-spouse,Adm-clerical,Husband,White,Male,0,0,40,United-States,<=50K


### From csv

In [5]:
# df=pd.read_csv(pathgreatdgpt2+'.csv')
# df['income'] = df['income'].apply(lambda x: x.strip())
# df.head()

In [6]:
real = pd.read_csv(pathreal)
real = real[df.columns]
ords = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country', 'income']
ordvals = {col:set(real[col].unique()) for col in ords}
for col in ordvals:
    ordvals[col] = [val.strip() for val in ordvals[col]]
# ordvals

In [7]:
for col in ordvals:
    df = df[df[col].isin(ordvals[col])]
    print(col, len(df))
# df[[df[col].isin(ordvals[col]) for col in ordvals]]
df
# df=df[df['workclass'].isin(ordvals['workclass']) & df['income'].isin(ordvals['income'])]
# df.head()

workclass 9293
education 9251
marital-status 9225
occupation 9164
relationship 9152
race 9134
sex 9132
native-country 9110
income 9106


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,53,Private,281231,Masters,14,Married-civ-spouse,Exec-managerial,Husband,White,Male,7298,0,60,Mexico,>50K
1,30,Private,328764,Masters,14,Divorced,Prof-specialty,Unmarried,White,Female,0,0,55,Italy,>50K
2,41,Private,185835,Doctorate,16,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,35,United-States,>50K
3,24,Private,112156,Some-college,10,Never-married,Adm-clerical,Own-child,White,Female,0,0,40,United-States,<=50K
4,25,Private,108425,HS-grad,9,Married-civ-spouse,Adm-clerical,Husband,White,Male,0,0,40,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9315,51,Private,282913,Bachelors,13,Never-married,Prof-specialty,Not-in-family,White,Male,0,0,40,United-States,<=50K
9316,49,Private,256091,Doctorate,16,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,70,United-States,>50K
9317,54,Private,204940,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,>50K
9318,39,Local-gov,374072,Masters,14,Separated,Prof-specialty,Unmarried,Asian-Pac-Islander,Male,0,0,45,Philippines,>50K


In [8]:
df.to_csv(path+'clean.csv', index=False)